<a href="https://colab.research.google.com/github/PatoJosefo/API-NEXUS/blob/main/filtro_cargas_por_NCM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output

In [ ]:
drive.mount('/content/drive')
%cd "/content/drive/MyDrive/Colab Notebooks/API1S/base_dados_por_NCM"

In [ ]:
df_ncm = pd.read_csv("NCM.csv", sep=";", encoding="latin1", on_bad_lines="skip", usecols="CO_NCM NO_NCM_POR".split())
df_via = pd.read_csv("VIA.csv", sep=";", encoding="latin1", on_bad_lines="skip", usecols="CO_VIA NO_VIA".split())
df_pais = pd.read_csv("PAIS.csv", sep=";", encoding="latin1", on_bad_lines="skip", usecols="CO_PAIS NO_PAIS".split())

arquivos_ncm_exp = []
arquivos_ncm_imp = []

for ano in range(2019, 2026):
    colunas = ["CO_NCM", "SG_UF_NCM", "CO_VIA", "KG_LIQUIDO", "VL_FOB", "CO_MES", "CO_PAIS"]

    df_exp = pd.read_csv(f"EXP_{ano}.csv", sep=";", encoding="latin1", on_bad_lines="skip", usecols=colunas)
    df_exp = df_exp[df_exp['SG_UF_NCM'] == 'SP']
    df_exp = pd.merge(df_exp, df_ncm, on="CO_NCM", how="left")
    df_exp = pd.merge(df_exp, df_via, on="CO_VIA", how="left")
    df_exp = pd.merge(df_exp, df_pais, on="CO_PAIS", how="left")
    arquivos_ncm_exp.append(df_exp)

    df_imp = pd.read_csv(f"IMP_{ano}.csv", sep=";", encoding="latin1", on_bad_lines="skip", usecols=colunas)
    df_imp = df_imp[df_imp['SG_UF_NCM'] == 'SP']
    df_imp = pd.merge(df_imp, df_ncm, on="CO_NCM", how="left")
    df_imp = pd.merge(df_imp, df_via, on="CO_VIA", how="left")
    df_imp = pd.merge(df_imp, df_pais, on="CO_PAIS", how="left")
    arquivos_ncm_imp.append(df_imp)


In [ ]:
def formatar_valor_reais(valor_reais):
  if valor_reais >= 1_000_000_000:
    return f"R$ {valor_reais / 1_000_000_000:.2f} bi"
  elif valor_reais >= 1_000_000:
    return f"R$ {valor_reais / 1_000_000:.2f} mi"
  elif valor_reais >= 1_000:
    return f"R$ {valor_reais / 1_000:.2f} mil"
  else:
    return f"R$ {valor_reais:.2f}"

In [ ]:
print('\nCampos obrigatórios: \n')

ncm_input = widgets.IntText(description="Código NCM:")

anos_dropdown = widgets.Dropdown(
    options=[2019, 2020, 2021, 2022, 2023, 2024, 2025],
    value=2019,
    description='Ano:'
)

atividade_dropdown = widgets.Dropdown(
    options=['Exportação', 'Importação'],
    value='Exportação',
    description='Atividade:'
)

tipo_dropdown = widgets.Dropdown(
    options=['FOB', 'Valor Agregado', 'KG líquido'],
    value='FOB',
    description='Tipo:'
)
display(ncm_input,anos_dropdown, atividade_dropdown, tipo_dropdown)

print('\nCampos opcionais: \n')

via_dropdown = widgets.Dropdown(
    options=[('Todos', None), ('Entrada/saída Ficta',10), ('Marítima',1), ('Fluvial',2), ('Lacustre',3),
             ('Aérea',4), ('Postal',5), ('Ferroviária',6), ('Rodoviária',7), ('Conduto/Rede de Transmissão',8),
             ('Meios próprios',9), ('Reboque',13), ('Courier',11), ('Vicinal Fronteiriço',15), ('Dutos',14),
             ('Em mãos',12), ('Via não declarada', 0), ('Via desconhecida',99)],
    value=None,
    description='Via:'
)

periodo_dropdown = widgets.Dropdown(
    options=['Mensal', 'Semestral'],
    value=None,
    description='Período:'
)
pais_dropdown = widgets.Dropdown(
    options=[(row['NO_PAIS'], row['CO_PAIS']) for _, row in df_pais.iterrows()],
    value=None,
    description='País:'
)
def atualizar_pais(mudar):
  global pais_selecionado
  global co_pais_selecionado
  if pais_dropdown.value is not None:
      pais_selecionado = df_pais[df_pais['CO_PAIS'] == pais_dropdown.value]['NO_PAIS'].values[0]
      co_pais_selecionado = pais_dropdown.value
  else:
      pais_selecionado = None
      co_pais_selecionado = None

pais_dropdown.observe(atualizar_pais, names='value')

display(periodo_dropdown, via_dropdown, pais_dropdown)

periodo_sem = widgets.Dropdown(
    options=['Janeiro-Junho', 'Julho-Dezembro'],
    value='Janeiro-Junho',
    description='Semestre:'
)

meses = widgets.Dropdown(
    options=[('Janeiro',1),('Fevereiro',2),('Março',3), ('Abril',4), ('Maio',5), ('Junho',6),('Julho',7), ('Agosto',8),
             ('Setembro',9), ('Outubro',10), ('Novembro',11), ('Dezembro',12)],
    value=1,
    description='Mês:'
)
output = widgets.Output()

In [ ]:
def atualizar_periodo(_):
  with output:
    clear_output()
    if periodo_dropdown.value == 'Semestral':
      display(periodo_sem)
    elif periodo_dropdown.value == 'Mensal':
      display(meses)

periodo_dropdown.observe(atualizar_periodo, names='value')

display(output)
atualizar_periodo(None)

In [ ]:
def realizar_analise(_):
  ano_selecionado = anos_dropdown.value
  atividade_selecionada = atividade_dropdown.value
  tipo_selecionado = tipo_dropdown.value
  cod_ncm = ncm_input.value

  indice = df_ncm[df_ncm['CO_NCM'] == cod_ncm].index[0]
  ncm_nome = df_ncm.loc[indice, 'NO_NCM_POR']

  cod_via = via_dropdown.value
  via_nome = next((n for n, v in via_dropdown.options if v == cod_via), "Todos")

  periodo = periodo_dropdown.value
  sem_selecionado = periodo_sem.value if periodo == 'Semestral' else None
  cod_mes = meses.value if periodo == 'Mensal' else None
  co_pais_filtro = co_pais_selecionado if pais_dropdown.value is not None else None
  mes_selecionado = dict(meses.options).get(cod_mes, None)
  periodo_selecionado = sem_selecionado if periodo == 'Semestral' else mes_selecionado

  indice = ano_selecionado - 2019
  arquivo_selecionado = arquivos_ncm_imp[indice] if atividade_selecionada == "Importação" else arquivos_ncm_exp[indice]

  base_filtro = arquivo_selecionado[arquivo_selecionado['CO_NCM'] == cod_ncm]

  if cod_via is not None:
    base_filtro = base_filtro[base_filtro['CO_VIA'] == cod_via]

  if periodo == "Mensal" and cod_mes is not None:
    base_filtro = base_filtro[base_filtro['CO_MES'] == cod_mes]
  elif periodo == "Semestral" and sem_selecionado is not None:
    codigo_sem = {'Janeiro-Junho': 1, 'Julho-Dezembro': 2}
    lista_sem = list(range(1, 7)) if codigo_sem[sem_selecionado] == 1 else list(range(7, 13))
    base_filtro = base_filtro[base_filtro['CO_MES'].isin(lista_sem)]
  if co_pais_filtro is not None:
      base_filtro = base_filtro[base_filtro['CO_PAIS'] == co_pais_filtro]

  match tipo_selecionado:
    case "FOB":
      soma_fob = base_filtro['VL_FOB'].sum()
      print(f'\nMercadoria: {ncm_nome}\n')
      print(f"Soma de FOB para o NCM {cod_ncm}: {formatar_valor_reais(soma_fob)}\nVia: {via_nome}\nPeríodo: {periodo_selecionado or 'Todos'}\nPaís: {pais_selecionado}\nAno: {ano_selecionado}")
    case "Valor Agregado":
      print(f'\nMercadoria: {ncm_nome}\n')
      base_filtro = base_filtro[base_filtro["KG_LIQUIDO"] > 0]
      base_filtro['VALOR_AGREGADO'] = base_filtro['VL_FOB'] / base_filtro['KG_LIQUIDO']
      soma_va = base_filtro['VALOR_AGREGADO'].sum()
      print(f"Soma de Valor Agregado para o NCM {cod_ncm}: {formatar_valor_reais(soma_va)}\nVia: {via_nome}\nPeríodo: {periodo_selecionado or 'Todos'}\nPaís: {pais_selecionado}\nAno: {ano_selecionado}")
    case "KG líquido":
      print(f'\nMercadoria: {ncm_nome}\n')
      soma_kgl = base_filtro['KG_LIQUIDO'].sum()
      print(f"Soma de KG Líquido para o NCM {cod_ncm}: {soma_kgl}kg\nVia: {via_nome}\nPeríodo: {periodo_selecionado or 'Todos'}\nPaís: {pais_selecionado}\nAno: {ano_selecionado}")

button = widgets.Button(description="Realizar Análise")
button.on_click(realizar_analise)
display(button)